# Initial Demo using Beer data

In [1]:
%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
sys.path.append('..')

In [3]:
from constants import FILE_FORMAT_MZML, FEATURE_SET_BINNED_005
from ms2lda.main import msfile_to_corpus
from ms2lda.lda_variational import VariationalLDA
from motifdb.main import get_motifset_list, post_motifsets, acquire_motifdb, FeatureMatcher
from ms2lda.loaders import LoadMZML
from ms2lda.feature_maker import MakeBinnedFeatures
from ms2lda.reporting import write_topic_report, write_motifs_in_scans

## 1. Feature Extraction

In [4]:
ms2_file = 'test_data/Beer_multibeers_1_T10_POS.mzML'
ms2_format = FILE_FORMAT_MZML

min_ms1_intensity = 0
min_ms2_intensity = 50
mz_tol = 5.0
rt_tol = 10.0
feature_set_name = FEATURE_SET_BINNED_005

K = 300
corpus_json = os.path.join('test_data', 'beer1pos.json')

In [5]:
lda_dict = msfile_to_corpus(ms2_file, ms2_format, min_ms1_intensity, min_ms2_intensity, mz_tol, rt_tol, feature_set_name, K, corpus_json=corpus_json)

2023-02-21 22:44:38.376 | INFO     | ms2lda.main:msfile_to_corpus:59 - Loading test_data/Beer_multibeers_1_T10_POS.mzML using mzML loader
2023-02-21 22:44:38.393 | INFO     | ms2lda.loaders:load_spectra:426 - Loading spectra from test_data/Beer_multibeers_1_T10_POS.mzML
2023-02-21 22:44:42.840 | INFO     | ms2lda.loaders:load_spectra:555 - Found 7565 ms2 spectra, and 252886 individual ms2 objects
2023-02-21 22:44:42.843 | INFO     | ms2lda.main:msfile_to_corpus:65 - bin_width = 0.005000
2023-02-21 22:44:42.844 | INFO     | ms2lda.main:msfile_to_corpus:68 - Using Binning feature creator with bin_width = 0.005 to make features
2023-02-21 22:44:50.227 | INFO     | ms2lda.feature_maker:make_features:141 - 7562 documents
2023-02-21 22:44:50.227 | INFO     | ms2lda.feature_maker:make_features:142 - After removing empty words, 54765 words left
2023-02-21 22:44:50.258 | INFO     | ms2lda.main:msfile_to_corpus:88 - Saving lda_dict to test_data/beer1pos.json


In [6]:
lda_dict.keys()

dict_keys(['corpus', 'word_index', 'doc_index', 'doc_metadata', 'topic_index', 'topic_metadata', 'features'])

## 2. Run LDA

Runs standard Variational LDA with no added motifs

In [7]:
n_its = 10
corpus = lda_dict['corpus']
features = lda_dict['features']
doc_metadata = lda_dict['doc_metadata']

In [8]:
vlda = VariationalLDA(corpus=corpus, K=K, normalise=1000.0)
vlda.run_vb(n_its=n_its, initialise=True)
vd = vlda.make_dictionary(
    features=features, metadata=doc_metadata, filename='test_data/beer.dict')

diff=30.348842: 100%|███████████████████████████████████████████████████████████████████| 10/10 [01:51<00:00, 11.17s/it]
2023-02-21 22:46:45.164 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 500
2023-02-21 22:46:45.286 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 1000
2023-02-21 22:46:45.410 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 1500
2023-02-21 22:46:45.529 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 2000
2023-02-21 22:46:45.654 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 2500
2023-02-21 22:46:45.777 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 3000
2023-02-21 22:46:45.883 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 3500
2023-02-21 22:46:45.993 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 4000
2023-02-21 22:46:46.108 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 4500
2023-02-21 22:46:46.188 | INFO     | ms2lda.lda_vari

## 3. MotifDB

A demonstration on how to retrieve motifsets from MotifDB and use them for LDA inference.

Below sections are based on https://github.com/glasgowcompbio/pySubstructures/blob/master/scripts/ms2lda_runfull.py. 
This is a copy of the script currently used in GNPS now to run https://ccms-ucsd.github.io/GNPSDocumentation/ms2lda. 

### a. Get the list of motif sets available from the server.

In [9]:
motifset_dict = get_motifset_list()
motifset_dict

{'Urine derived Mass2Motifs': 1,
 'GNPS library derived Mass2Motifs': 2,
 'Massbank library derived Mass2Motifs': 4,
 'Rhamnaceae Plant Mass2Motifs': 5,
 'Euphorbia Plant Mass2Motifs': 3,
 'LDB_NEG_MotifDB_02': 33,
 'MIADB_pos_indole': 29,
 'MIADB_pos_60': 30,
 'MIADB_pos_100': 31,
 'Streptomyces S29': 32,
 'fragment_253.1925': 34,
 'Tps6_motifs': 36,
 'LDB_NEG_MotifDB_01': 17,
 'MIADB_pos': 18,
 'Streptomyces and Salinispora Mass2Motifs': 6,
 'Photorhabdus and Xenorhabdus Mass2Motifs': 16,
 'LDB MotifDB POS': 37,
 'Planomonospora-associated Mass2Motifs': 38,
 'Human Urine ESI_POS': 39,
 '20210810_16HT': 40,
 'cholestane_glicoalkaloids': 64,
 'Test25': 72,
 'Aporphinic_alkaloids': 73,
 'Urine derived Mass2Motifs 2': 74,
 'NP_rings': 78,
 'Test_FA': 75,
 'Test_FA2': 76,
 'DHA_motif': 77,
 'test': 79}

Below are the choices of motifsets available when running from GNPS. I think it's a series of checkboxes there.

In [10]:
selected_motifsets = [
    'GNPS library derived Mass2Motifs',
    'Massbank library derived Mass2Motifs',
    'Urine derived Mass2Motifs',
    'Euphorbia Plant Mass2Motifs',
    'Rhamnaceae Plant Mass2Motifs',
    'Streptomyces and Salinispora Mass2Motifs',
    'Photorhabdus and Xenorhabdus Mass2Motifs',
]
motifset_ids = [motifset_dict[m] for m in selected_motifsets]
motifset_ids

[2, 4, 1, 3, 5, 6, 16]

In GNPS, user has the option to also specify their own ms2lda experiment id on top of the motifsets above

In [11]:
user_motifset_ids = [] # leave blank for now
db_list = list(set(motifset_ids + user_motifset_ids))
db_list

[1, 2, 3, 4, 5, 6, 16]

### b. Acquire motifset from MS2LDA.org.

**TODO: modify so this reads locally stored file rather than fetching from server.** Not sure if we need to, but we can also use https://github.com/glasgowcompbio/pySubstructures/blob/master/scripts/extract_motifs_from_server.py to get all motifs on the server.

Make a POST request to the server to fetch the specified motif spectra and metadata. 

`motifdb_spectra` and `motifdb_metadata` are dictionaries where the key is the motif name, and the value is another dictionary.

In [12]:
motifdb_spectra, motifdb_metadata, motifdb_features = acquire_motifdb(db_list)

/opt/anaconda3/envs/pySub/lib/python3.9/site-packages/urllib3/connectionpool.py:1043: InsecureRequestWarning:

Unverified HTTPS request is being made to host 'ms2lda.org'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings



In [13]:
len(motifdb_spectra), len(motifdb_metadata), len(motifdb_features)

(389, 389, 15418)

In [14]:
type(motifdb_spectra), type(motifdb_metadata), type(motifdb_features)

(dict, dict, set)

The value in `motifdb_spectra` is another dictionary, where the key is the feature name, and the value is the probability (maybe? i forgot .. to check)

In [15]:
example = 'StrepSalini_motif_10.m2m'
motifdb_spectra[example]

{'loss_29.9500': 0.0037582452870463,
 'loss_49.8500': 0.00106985503922065,
 'fragment_587.1500': 0.00154228226008414,
 'fragment_529.2500': 0.00130648997499644,
 'fragment_659.2500': 0.00156497912260926,
 'fragment_729.3500': 0.00106085240137599,
 'fragment_673.3500': 0.00207616089295373,
 'fragment_930.4500': 0.00383889422395102,
 'fragment_569.5500': 0.00282630642220952,
 'fragment_864.7500': 0.00129509734421731,
 'fragment_551.2500': 0.00321410138141022,
 'fragment_539.2500': 0.0128525126605606,
 'fragment_842.3500': 0.00115560481154524,
 'fragment_752.8500': 0.00110374498496376,
 'fragment_620.7500': 0.00118106808757062,
 'fragment_724.3500': 0.0064725022999416,
 'fragment_546.7500': 0.00117142048046988,
 'fragment_869.4500': 0.00138508685008911,
 'fragment_827.3500': 0.00120619058801461,
 'fragment_500.2500': 0.00208944869536447,
 'fragment_428.1500': 0.00380772533224174,
 'loss_93.1500': 0.00294927959215106,
 'fragment_725.3500': 0.00902286761226882,
 'fragment_881.5500': 0.00106

The value in `motifdb_metadata` is simply a dictionary of metadata for this motif.

In [16]:
motifdb_metadata[example]

{'short_annotation': 'Actinomycin peptide lactone related Mass2Motif',
 'type': 'learnt',
 'annotation': 'Actinomycin related Mass2Motif (H-VPMeGMeV-OH peptide lactone sequence)',
 'name': 'motif_10',
 'motifdb_id': 171208,
 'motifdb_url': 'http://ms2lda.org/motifdb/motif/171208'}

And `motifdb_features` is a set of features used in all motifs above.

In [17]:
'loss_29.9500' in motifdb_features

True

## 4. Combined workflow used in GNPS

### a. Run feature extraction following the ms2lda_runfull.py script.

**TODO: a lot of code duplications with msfile_to_corpus function above**

**TODO: in GNPS we are able to take a peaklist CSV file to use for matching (filtering) of the extrated MS1 peaks from the fragmentation mzML file.**

In [18]:
# in GNPS we only accept MGF as input (since that's the output from molecular networking), not mzML
ms2_file = 'test_data/Beer_multibeers_1_T10_POS.mzML'
ms2_format = FILE_FORMAT_MZML

# same values as what's used in GNPS now
min_ms1_intensity = 0
min_ms2_intensity = 50 
input_bin_width = 0.005

peaklist = None # not working for now

In [19]:
loader = LoadMZML(mz_tol=mz_tol,
                  rt_tol=rt_tol, peaklist=peaklist,
                  min_ms1_intensity=min_ms1_intensity,
                  min_ms2_intensity=min_ms2_intensity)
ms1, ms2, metadata = loader.load_spectra([ms2_file])

2023-02-21 22:48:03.493 | INFO     | ms2lda.loaders:load_spectra:426 - Loading spectra from test_data/Beer_multibeers_1_T10_POS.mzML
2023-02-21 22:48:08.303 | INFO     | ms2lda.loaders:load_spectra:555 - Found 7565 ms2 spectra, and 252886 individual ms2 objects


In [20]:
m = MakeBinnedFeatures(
    bin_width=input_bin_width)
corpus, features = m.make_features(ms2)

2023-02-21 22:48:15.717 | INFO     | ms2lda.feature_maker:make_features:141 - 7562 documents
2023-02-21 22:48:15.717 | INFO     | ms2lda.feature_maker:make_features:142 - After removing empty words, 54765 words left


In [21]:
first_key = list(corpus.keys())[0]
first_key

'Beer_multibeers_1_T10_POS.mzML'

In [22]:
corpus = corpus[first_key]
len(corpus)

7562

### b. Perform feature matching from MotifDB features to the corpus features

In [23]:
fm = FeatureMatcher(motifdb_features, features)
motifdb_spectra = fm.convert(motifdb_spectra)

Finished matching (fragment). 3506 exact matches, 342 overlap matches, 6787 new features
Finished matching (loss). 3853 exact matches, 311 overlap matches, 619 new features


In [24]:
# Add the motifdb features to avoid problems when loading the dict into vlda later
bin_width = m.bin_width
added = 0
for f in motifdb_features:
    if not f in features:
        word_mz = float(f.split('_')[1])
        word_mz_min = word_mz - bin_width / 2
        word_mz_max = word_mz + bin_width / 2
        features[f] = (word_mz_min, word_mz_max)
        added += 1

In [25]:
print("Added {} features".format(added))

Added 8059 features


### c. Run LDA inference for the fixed + free topics

In [26]:
K = 100  # number of *new* topics
vlda = VariationalLDA(corpus, K=K, normalise=1000.0,
                      fixed_topics=motifdb_spectra,
                      fixed_topics_metadata=motifdb_metadata)

In [27]:
input_iterations = 10

# note that for real runs the number of iterations is recommended to be 1000 or higher
vlda.run_vb(initialise=True, n_its=input_iterations)

diff=12.206977: 100%|███████████████████████████████████████████████████████████████████| 10/10 [02:35<00:00, 15.52s/it]


### d. Save a dict file that can be uploaded to ms2lda.org for analysis later

In [28]:
vd = vlda.make_dictionary(features=features, metadata=metadata, filename='test.dict')

2023-02-21 22:50:58.428 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 500
2023-02-21 22:50:58.602 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 1000
2023-02-21 22:50:58.783 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 1500
2023-02-21 22:50:58.924 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 2000
2023-02-21 22:50:59.059 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 2500
2023-02-21 22:50:59.194 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 3000
2023-02-21 22:50:59.309 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 3500
2023-02-21 22:50:59.431 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 4000
2023-02-21 22:50:59.553 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 4500
2023-02-21 22:50:59.630 | INFO     | ms2lda.lda_variational:make_dictionary:442 - Done 5000
2023-02-21 22:50:59.709 | INFO     | ms2lda.lda_variational:make_dictionary:442 -

### e. MolNet Integration

Write a network graph file that can be loaded to Cytoscape later on

In [29]:
# from ms2lda.molnet_integration import write_output_files

# # not sure what should be the values of all these parameters
# pairs_file = None 
# output_prefix = None

# write_output_files(vd, pairs_file, output_prefix, metadata,
#                    overlap_thresh=args.input_network_overlap, p_thresh=args.input_network_pvalue,
#                    X=args.input_network_topx, motif_metadata=motifdb_metadata)

### f. Generate PDF report

In [35]:
# Writing the report - ntoe that you might need to set the 'backend' argument
# for this method to work (see the method in lda.py) as it depends what on
# your system will render the pdf...

output_prefix = 'test'
write_topic_report(vd, output_prefix + '_topic_report.pdf')

  0%|          | 0/489 [00:00<?, ?it/s]

In [31]:
output_prefix = 'test'
overlap_thresh = 0.2
p_thresh = 0.1
X = 5
write_motifs_in_scans(vd, metadata, overlap_thresh, p_thresh, X, motifdb_metadata, output_prefix)